In [2]:
import numpy as np
from astropy.io import fits
from marvin.tools import Maps
from tqdm import tqdm
import Bicone_Classifier
import multiprocessing as mp
import copy


[INFO]: No release version set. Setting default to DR17
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: Passing field metadata as keyword arguments is deprecated. Use the explicit `metadata=...` argument instead. Additional metadata: {'location': 'view_args'} (RemovedInMarshmallow4Warning)
[WARNING]: The 'missing' argument to fields is deprecated. Use 'load_default' instead. (RemovedInMarshmallow4Warning)


HEADER_PATH = '/Users/runquanguan/Documents/dapall-v3_1_1-3.1.0.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']

In [2]:
HEADER_PATH = '/Users/runquanguan/Documents/bicone_training_set.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']


In [9]:
np.linspace(0.6, 4, 18)

array([0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4, 2.6, 2.8, 3. ,
       3.2, 3.4, 3.6, 3.8, 4. ])

In [3]:
def first_filter(data):
    
    if len(data) >= 2:
        try:

            ellip = Bicone_Classifier.ellip_gen(data)
            loss_list = []

            for i in np.linspace(0.6, 4, 8):
                start, end = round(i,1), round(i,1)+0.3
                curve = Bicone_Classifier.ellip_ring_curve(ellip, in_r = start, out_r = end, cycle = 2)
                result = Bicone_Classifier.fourier_classifier(curve)

                if result[1][1] == 3:
                    loss_list.append(result[2])
                else:
                    loss_list.append(np.array([0]))

            if sum(loss_list) >= 220:
                return data
        except:
            pass
            
    else:
        pass

In [4]:
if __name__ == '__main__':
    pool = mp.Pool(processes=4)

    results = []
    for result in pool.imap_unordered(first_filter, plateifu):
        results.append(result)
       
    print(results)

['7957-6102', '7958-9101', '7960-6103', '7963-3703', '7977-3702', '7972-3704', '7990-3702', '7990-1901', '7990-9102', '7991-6104', '8077-6102', '8080-3704', '8081-1901', None, '8083-3704', '8138-6101', '8247-6104', '8248-3701', '8245-3704', '8156-6102', '8249-3702', '8250-6104', '8252-9101', '8253-12703', '8261-6104', '8309-12701', '8311-12704', '8320-1901', '8322-3704', '8324-6103', '8324-6104', '8324-12701', '8338-3704', '8439-3701', '8441-3703', '8338-6101', '8445-9102', None, '8448-3701', '8453-1901', None, '8461-1901', '8458-12701', None, '8462-9101', '8466-1901', '8465-12704', '8486-12705', '8550-3701', '8551-1901', '8550-3703', '8551-6104', '8555-1901', '8561-3703', '8562-1901', '8566-3701', '8568-1901', '8588-1902', '8588-3703', '8592-3702', '8597-1901', None, '8612-12704', '8610-12703', '8614-1902', None, '8715-6103', '8626-12702', '8716-9101', '8723-1902', None, '8718-12702', '8726-1901', None, '8938-1902', '8940-6101', '8946-1901', '8947-3701', '8950-12701', '8950-12705', '8

In [5]:
result_copy = copy.deepcopy(results)

first_stage = []

for i in result_copy:
    if i != None:
        first_stage.append(i)
        
len(first_stage)

141

index_list = []
for i in first_stage:
    index_list.append(float(i[1]))
    
index_copy = copy.deepcopy(index_list)
index_copy.sort()
for i in index_copy:
    if i <= 220:
        index = index_list.index(i)
        plateifu = first_stage[index][0]
        print(plateifu)
        
        
'''
Below 220:
11832-1902
8132-1902
8449-1902
8458-1901
9501-1902
9091-12703
11834-9102
8619-9102
11863-6103
8936-1902
8602-6101
8716-12704
8461-1902
'''

In [6]:
def second_filter(data):        
    
        ellip = Bicone_Classifier.ellip_gen(data)
        axis_loss = []
        
        for i in np.linspace(0.6, 2, 8):
            
            start, end = round(i,1), round(i,1)+0.3
            curve = Bicone_Classifier.ellip_ring_curve(ellip, in_r = start, out_r = end, cycle = 2)
            result = Bicone_Classifier.fourier_classifier(curve)

            axis_loss.append(result[4])

        total_axis_loss = sum(axis_loss)

        if total_axis_loss <= 225:
            return data
        else:
            pass


In [7]:
if __name__ == '__main__':
    pool = mp.Pool(processes=4)

    final_result = []
    for result in pool.imap_unordered(second_filter, first_stage):
        final_result.append(result)
       
    print(final_result)

['7960-6103', '7963-3703', '7958-9101', '7957-6102', '7977-3702', '7972-3704', None, '7990-1901', '7990-9102', '7991-6104', '8080-3704', '8077-6102', '8081-1901', '8083-3704', '8138-6101', '8247-6104', '8248-3701', '8245-3704', '8156-6102', '8249-3702', '8250-6104', '8252-9101', '8253-12703', '8261-6104', '8309-12701', None, '8311-12704', '8324-6103', None, '8324-6104', '8324-12701', '8338-3704', '8439-3701', '8441-3703', '8338-6101', '8445-9102', '8448-3701', '8453-1901', '8461-1901', '8458-12701', '8462-9101', '8466-1901', None, '8486-12705', '8550-3701', None, '8550-3703', '8551-6104', '8555-1901', '8561-3703', '8562-1901', '8566-3701', None, '8588-1902', None, '8597-1901', '8592-3702', '8612-12704', '8610-12703', None, '8715-6103', '8716-9101', '8626-12702', None, None, '8938-1902', '8718-12702', '8940-6101', None, '8947-3701', '8950-12701', '8950-12705', '8987-9102', '9030-1902', '8998-6102', '9001-12702', '9031-3702', '9040-1901', None, '9031-12704', '9090-1901', '9093-12701', '9

In [8]:
final_copy = copy.deepcopy(final_result)

second_stage = []

for i in final_copy:
    if i != None:
        second_stage.append(i)
        
len(second_stage)

123

In [18]:
c1 = fits.Column(name='plateifu', array=np.array(second_stage), format='20A')
t = fits.BinTableHDU.from_columns([c1])
t.writeto('bicone_candidates_AUTO_v2.fits')

# Check the missed galaxies

In [9]:
## Test Section:

### Save missing galaxies
missing = []

for i in plateifu:
    if i in second_stage:
        pass
    else:
        missing.append(i)
    


In [13]:
len(missing)

33

## Rerun the pipeline and make sure there's no bug for missing

In [ ]:
def missing_print(data):
    
    if len(data) >= 2:
        try:

            ellip = Bicone_Classifier.ellip_gen(data)
            loss_list = []
            axis_loss = []

            for i in np.linspace(0.6, 2, 8):
                start, end = round(i,1), round(i,1)+0.3
                curve = Bicone_Classifier.ellip_ring_curve(ellip, in_r = start, out_r = end, cycle = 2)
                result = Bicone_Classifier.fourier_classifier(curve)
                
                axis_loss.append(result[4])

                if result[1][1] == 3:
                    loss_list.append(result[2])
                else:
                    loss_list.append(np.array([0]))

            
            total_axis_loss = sum(axis_loss)
            
            return data, sum(loss_list), total_axis_loss
        
        
        except:
            pass
            
    else:
        pass
    
    

In [16]:

if __name__ == '__main__':
    pool = mp.Pool(processes=4)

    results = []
    for result in pool.imap_unordered(missing_print, missing):
        results.append(result)
       
    print(results)

[('8132-1902', array([120.29635673]), 328), ('8320-1901', array([374.82057521]), 333), ('7990-3702', array([505.86751378]), 291), ('8322-3704', 629.7166339757821, 272), ('8449-1902', array([160.07165649]), 226), ('8458-1901', array([170.69310421]), 419), ('8461-1902', array([214.9286914]), 299), ('8551-1901', array([605.35386355]), 303), ('8588-3703', array([351.00440214]), 273), ('8568-1901', array([382.97824888]), 342), ('8465-12704', 955.5371670836423, 237), ('8602-6101', array([199.31538551]), 265), ('8614-1902', array([519.60117591]), 262), ('8619-9102', array([185.15543105]), 125), ('8716-12704', array([204.42536548]), 123), ('8726-1901', array([479.79853398]), 233), ('8723-1902', array([547.57183727]), 266), ('8946-1901', array([526.22502395]), 234), ('8936-1902', array([197.45295077]), 234), ('9089-3704', array([266.72655436]), 253), ('9485-3701', array([302.88490405]), 312), ('9091-12703', array([175.18964128]), 210), ('9501-1902', array([172.3887032]), 223), ('10507-1901', ar

In [22]:
missing_print_copy = copy.deepcopy(results)


ints_loss = []
axis_loss = []

for i in missing_print_copy:
    ints_loss.append(float(i[1]))
    axis_loss.append(float(i[2]))
    

In [33]:
len(missing_print_copy)

33

In [32]:
# criteria:
#   ints_loss >= 220
#   axis_loss <= 225

for i in missing_print_copy:
    if float(i[1]) >= 220 and float(i[2]) <= 225:
        print(i[0])

# Combine Two Filters

In [ ]:
def bicone_selection(data):
    
    if len(data) >= 2:
        try:

            ellip = Bicone_Classifier.ellip_gen(data)
            ints_index = []
            axis_loss = []

            for i in np.linspace(0.6, 2, 8):
                start, end = round(i,1), round(i,1)+0.3
                curve = Bicone_Classifier.ellip_ring_curve(ellip, in_r = start, out_r = end, cycle = 2)
                result = Bicone_Classifier.fourier_classifier(curve)
                
                axis_loss.append(result[4])

                if result[1][1] == 3:
                    ints_index.append(result[2])
                else:
                    ints_index.append(np.array([0]))

            total_ints_loss = sum(ints_index)
            total_axis_loss = sum(axis_loss)
            
            if total_ints_loss >= 220 and total_axis_loss <= 225:
                return data
            else:
                pass
        
        
        except:
            pass
            
    else:
        pass
    
   

In [ ]:
if __name__ == '__main__':
    pool = mp.Pool(processes=4)

    results = []
    for result in pool.imap_unordered(bicone_selection, plateifu):
        results.append(result)
       
    print(results)